In [31]:
import sys
sys.executable

'/homes/gws/jc383/WindowsFolders/Desktop/DrosSag/bin/python3'

In [43]:
from scipy.stats import spearmanr
import pandas as pd
import numpy as np
import logging
import torch
import csv
import sys
import os
import anndata as ad

sys.path.append(os.path.join(sys.path[0], '../'))
from data_utils import utils

columns_metadata = pd.read_csv("../data_utils/data/static/GSE190147_scirnaseq_gene_matrix.columns.csv", delimiter="\t")
rows_metadata = pd.read_csv("../data_utils/data/static/GSE190147_scirnaseq_gene_matrix.rows.csv", delimiter="\t")
gene_expression_matrix = pd.read_csv("../data_utils/data/static/GSE190147_scirnaseq_gene_matrix.txt", delimiter=" ")
gene_expression_matrix_file = "../data_utils/data/static/GSE190147_scirnaseq_gene_matrix.txt"
columns_metadata_file = "../data_utils/data/static/GSE190147_scirnaseq_gene_matrix.columns.csv"
rows_metadata_file = "../data_utils/data/static/GSE190147_scirnaseq_gene_matrix.rows.csv"
meta = pd.read_csv("../data_utils/data/static/meta.csv")


/tmp/ipykernel_1544300/913425730.py:20: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv("../data_utils/data/static/meta.csv")


In [33]:
# general setup
selected_clusters = [5, 6, 7, 8, 9]
selected_time_points = ['hrs_00_02', 'hrs_01_03', 'hrs_02_04', 'hrs_03_07', 'hrs_04_08']
samples_per_time_point = 4

In [35]:
# first for train set
utils.filter_by_seurat_cluster(meta, selected_clusters, selected_time_points, samples_per_time_point, '12311130_train_meta.csv', set())

# this is the big one
# believe this one barely hiccupped
filt_meta_df = pd.read_csv('12311130_train_meta.csv')
utils.filter_expr(columns_metadata, gene_expression_matrix, filt_meta_df, 'exp1_filtered_matrix.csv')

In [51]:
# only want to do once, put in entire matrix file
# THIS ONE LITERALLY TAKES FOREVERRRR
def create_genes_list(filtered_from_ordered_meta, rows_metadata, output_file, header):
    result_data = []
    with open(filtered_from_ordered_meta, "r") as matrix_file:
        csv_reader = csv.reader(matrix_file)
        if header:
            next(csv_reader)
            next(csv_reader)
        counter = 0
        for line in csv_reader:
            try:
                if counter == 0:
                    print(line)
                counter += 1
                row_idx, col_idx, value = [int(l) for l in line[0].split()]
                if counter == 1:
                    print(row_idx)
                    print(col_idx)
                    print(value)
                    counter += 1
                        
            except ValueError as ve:
                logging.error(f"Error parsing line: {line} - {ve}")
                continue
            gene_name = rows_metadata.iloc[row_idx - 2, 0]
            result_data.append([gene_name])
            
    result_df = pd.DataFrame(result_data, columns=['gene_name'])

    result_df.to_csv(output_file, index=False)
create_genes_list(gene_expression_matrix_file, rows_metadata, 'GENES_LIST_FINAL.csv', True)
utils.create_gene_dict('GENES_LIST_FINAL.csv', 'GENES_MAPPING_FINAL.txt')

['93 1 1']
93
1
1


NameError: name 'gene_names' is not defined

In [52]:
utils.create_gene_dict('GENES_LIST_FINAL.csv', 'GENES_MAPPING_FINAL.txt')

Taf12


In [53]:
# last one! deal with ys
utils.map_cell_id('12311130_train_meta.csv', 'seurat_clusters_dict.txt')

In [62]:
# now for test set
df = pd.read_csv('../data_utils/data/exp1/12311130_train_meta.csv')
cell_id = set(df['cell'])
print(cell_id)
print(len(cell_id))

utils.filter_by_seurat_cluster(meta, selected_clusters, selected_time_points, samples_per_time_point, '12311130_test_meta.csv', cell_id)

filt_test_meta = pd.read_csv('12311130_test_meta.csv')
utils.filter_expr(columns_metadata, gene_expression_matrix, filt_test_meta, 'exp1_filtered_matrix_TEST.csv')

{'exp10_idx1AATACGATAA_idx2AACTTATAAG.GTAAGCTCTCGACGCGACT', 'exp10_idx1TTGCGAGCAT_idx2CGACTACCAA.TCCTATAAGTAATCGATAAT', 'exp10_idx1AATACGATAA_idx2AACTTATAAG.CGTACCTATGAACTGCATC', 'exp2_idx1AGACCATATC_idx2TGCCAGATGG.CCGTCGATTTATCATGATC', 'exp11_idx1AGTCTATCGT.TTGGAGTTATTGATACGAGT', 'exp10_idx1ACGGCGTTAA_idx2ACCGTTACGA.TATGAGAACTTCCTACCAGT', 'exp4_idx1GGTTATCGAA_idx2GTTCCGACGA.TAGATGACTTTCCGGCCTCG', 'exp12_idx1CGTCGAGCGC.ATATGGATCTTTGGACGCTG', 'exp1_idx1GCGCGTTCAT_idx2CCTATAAGCT.TTGGCAATATGGCCAGTCCG', 'exp11_idx1TCGCCGCCGG.TTCTCATTGTAGGCCGCTCG', 'exp11_idx1TCTTGAGGTT.AACGAGCGTTCGCTAAGCA', 'exp1_idx1GTAGCAGCTA_idx2CTAACTCCAA.TTCGCATAATTGCGCGATGC', 'exp10_idx1AGTCCGCTGC_idx2GTTGACGGCC.GGAGCTCATTCCTACCAGT', 'exp11_idx1ATCTACCAGT.ATGGTAACTTTCGATATTGA', 'exp10_idx1GATTCTTGAA_idx2CAAGATTAAG.GTCATCTGTGATACGGAAC', 'exp10_idx1AATACGATAA_idx2AACTACCGAC.GGTTCGAGTGGTATGCTTG', 'exp1_idx1GTAGGCTGCA_idx2TGAGGTAGAA.CGAGATGCTTATCGTCGGC', 'exp10_idx1AATACGATAA_idx2ACGTTCAACC.TTATAGGCATCTCGATGGAG', 'exp11_

In [63]:
# ok actually last one
utils.map_cell_id('12311130_test_meta.csv', 'seurat_clusters_dict_TEST.txt')